In [30]:
import sklearn
import netCDF4
import numpy as np
import matplotlib.pyplot as plt

In [4]:
temp_data = netCDF4.Dataset("data/erai_t2m_daily.nc")  # 'temp' as in 'temperature'

In [173]:
sep = 2

def get_fine_data(t):
    return np.array(temp_data.variables["t2m"][t, ::-1, :])

def get_coarse_data(t):
    return np.array(temp_data.variables["t2m"][t, ::-1 * sep, ::sep])


def plot_data(data, cmap=plt.get_cmap('jet')):
    plt.pcolormesh(data, cmap=cmap)
    
    
def rms(p_data, f_data):
    return np.sqrt(np.sum((p_data - f_data) ** 2))

f_shape = get_fine_data(0).shape

In [174]:
def zero_func(x, y, c_data):
    if not y % sep and not x % sep:  # Known data
        return c_data[y // sep, x // sep]
    else:
        return 0
    
def mean_func(x, y, c_data):
    if not y % sep and not x % sep:  # Known data
        return c_data[y // sep, x // sep]
    elif y % sep and not x % sep:
        return 0.5 * (c_data[(y - 1) // 2, x // 2] + c_data[(y + 1) // 2, x // 2])
    elif not y % sep and x % sep:
        return 0.5 * (c_data[y // 2, (x - 1) // 2] + c_data[y // 2, (x + 1) // 2])
    else:
        return 0.25 * (c_data[(y - 1) // 2, (x - 1) // 2] + c_data[(y - 1) // 2, (x + 1) // 2] +
                       c_data[(y + 1) // 2, (x - 1) // 2] + c_data[(y + 1) // 2, (x + 1) // 2])

def local_interpolater(interpolate_func):
    def local_interpolater_inner(c_data):
        return_data = np.empty(f_shape, dtype=np.float32)
        for y in range(f_shape[0]):
            for x in range(f_shape[1]):
                return_data[y, x] = interpolate_func(x, y, c_data)
        return return_data
    return local_interpolater_inner

separate_data = local_interpolater(zero_func)
average_data = local_interpolater(mean_func)

In [158]:
fine_data = get_fine_data(0)
coarse_data = get_coarse_data(0)

In [153]:
averaged_data = average_data(coarse_data)
prepare_fine_data_for_average
rms(averaged_data, fine_data)

188.37268

In [162]:
sep = 2
temp_data.variables["t2m"][0, ::-1*sep, ::sep]

masked_array(
  data=[[256.4365234375, 256.421875, 256.408203125, ..., 256.45751953125,
         256.45068359375, 256.443359375],
        [257.400390625, 257.365234375, 257.330078125, ...,
         257.49267578125, 257.4619140625, 257.4306640625],
        [256.9931640625, 256.9287109375, 256.8642578125, ...,
         257.22509765625, 257.1474609375, 257.0703125],
        ...,
        [244.03663635253906, 243.95118713378906, 243.86622619628906, ...,
         244.11378479003906, 244.08741760253906, 244.06202697753906],
        [243.59425354003906, 243.55323791503906, 243.51319885253906, ...,
         243.68849182128906, 243.65724182128906, 243.62550354003906],
        [244.12794494628906, 244.11573791503906, 244.10450744628906, ...,
         244.17189025878906, 244.15724182128906, 244.14259338378906]],
  mask=[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
    

In [164]:
temp_data.variables["t2m"][0, ::-1, :]

masked_array(
  data=[[256.4365234375, 256.4296875, 256.421875, ..., 256.4462890625,
         256.443359375, 256.43994140625],
        [257.0439453125, 257.02978515625, 257.01611328125, ...,
         257.0634765625, 257.056640625, 257.05029296875],
        [257.400390625, 257.3828125, 257.365234375, ..., 257.4462890625,
         257.4306640625, 257.4150390625],
        ...,
        [243.81739807128906, 243.80226135253906, 243.78761291503906, ...,
         243.85450744628906, 243.84181213378906, 243.82960510253906],
        [244.12794494628906, 244.12208557128906, 244.11573791503906, ...,
         244.15040588378906, 244.14259338378906, 244.13526916503906],
        [244.40187072753906, 244.40040588378906, 244.39845275878906, ...,
         244.41358947753906, 244.40968322753906, 244.40577697753906]],
  mask=[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
      

In [170]:
 np.where(temp_data.variables["t2m"][0, ::-1, :] == 244.14259338378906)

(array([254], dtype=int64), array([510], dtype=int64))

In [171]:
x = temp_data.variables["t2m"][0]